In [ ]:
import os
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
from pathlib import Path

from scipy.stats import chi2_contingency, pearsonr
from statsmodels.stats.multitest import multipletests

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import __version__ as sklver
from xgboost import XGBClassifier
file_path = os.path.join("Data", "Hybrid_Augmented_TSAFE_Features.xlsx")
OUTDIR = Path("./empirical_eval_outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)
DPI = 600
N_TOP_PLOT = 20
SEED = 42


df = pd.read_excel(file_path)
if 'Order Date' in df.columns:
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df = df.sort_values(['Order Date']).reset_index(drop=True)

# 1.1) Ensure Plant_Destination (your new rule)
if 'Plant_Destination' not in df.columns:
    if {'Plant Code', 'Destination Port'}.issubset(df.columns):
        df['Plant_Destination'] = (
            df['Plant Code'].astype(str) + ' | ' + df['Destination Port'].astype(str)
        )
    else:
        raise ValueError("Cannot create 'Plant_Destination' because 'Plant Code' or 'Destination Port' is missing.")


original_features = [
    'Order ID','Order Date','Origin Port','Carrier','TPT','Service Level',
    'Ship ahead day count','Ship Late Day count','Customer','Product ID',
    'Plant Code','Destination Port','Unit quantity','Weight'
]
all_features = df.columns.tolist()
augmented_features = [c for c in all_features if c not in original_features]


features_of_interest_cat = [
    'Origin Port', 'Carrier', 'Plant Code', 'Destination Port', 'Plant_Destination'
]
features_of_interest_num = [
    'Unit quantity', 'Weight', 'TPT',
    'TPT_per_Unit', 'LeadTime_Deviation', 'Weight_per_Unit',
    'log_UnitQty', 'carrier_origin_risk', 'route_cum_late_rate',
    'route_bb_mean', 'carrier_bb_mean', 'route_orders_last7d',
    'route_roll10_Weight_q90', 'congestion_trend', 'Weight_vsCarrierMean',
    'seq_pos_norm'
]

features_of_interest_cat = [c for c in features_of_interest_cat if c in df.columns]
features_of_interest_num = [c for c in features_of_interest_num if c in df.columns]
features_of_interest = features_of_interest_cat + features_of_interest_num

print("Augmented features detected:", augmented_features)
print("Features of interest (present):", features_of_interest)


if 'Ship Late Day count' not in df.columns:
    raise ValueError("Missing 'Ship Late Day count' for target.")
y = (df['Ship Late Day count'] > 0).astype(int)


X_aug = df[augmented_features].copy()
X_aug = X_aug.replace([np.inf, -np.inf], np.nan)
X_aug_clean = X_aug.apply(lambda col: col.fillna(col.median()) if col.dtype != 'object' else col)


def cramers_v_from_chi2(chi2, n, k_rows, k_cols):
    # Bias-corrected Cramér’s V
    phi2 = chi2 / n
    r, k = k_rows, k_cols
    phi2corr = max(0, phi2 - (k-1)*(r-1)/(n-1))
    rcorr = r - (r-1)**2/(n-1)
    kcorr = k - (k-1)**2/(n-1)
    denom = min(kcorr-1, rcorr-1)
    return np.sqrt(phi2corr / denom) if denom > 0 else 0.0

stat_rows = []
for feat in augmented_features:
    s = X_aug_clean[feat]
    try:
        if s.dtype == 'object' or s.nunique() <= 10:
            tbl = pd.crosstab(s, y)
            chi2, p, dof, exp = chi2_contingency(tbl)
            V = cramers_v_from_chi2(chi2, n=len(df), k_rows=tbl.shape[0], k_cols=tbl.shape[1])
            stat_rows.append([feat, 'Chi-Square', chi2, p, V, 'Cramér_V'])
        else:
            valid = s.dropna().index
            r, p = pearsonr(s.loc[valid], y.loc[valid])
            stat_rows.append([feat, 'Point-Biserial (Pearson)', r, p, abs(r), '|r_pb|'])
    except Exception as e:
        stat_rows.append([feat, 'ERROR', np.nan, np.nan, np.nan, str(e)])

stat_df = pd.DataFrame(stat_rows, columns=['Feature', 'Test', 'Statistic', 'p_value', 'EffectSize', 'EffectType'])
mask_ok = stat_df['p_value'].notna()
stat_df.loc[mask_ok, 'q_value'] = multipletests(stat_df.loc[mask_ok, 'p_value'].values, method='fdr_bh')[1]
stat_df = stat_df.sort_values(['q_value', 'p_value'])


stat_df.to_excel(OUTDIR / "Empirical_Stats_Augmented.xlsx", index=False)
stat_df.to_csv(OUTDIR / "Empirical_Stats_Augmented.csv", index=False)
stat_focus = stat_df[stat_df['Feature'].isin(features_of_interest)].copy()
stat_focus.to_excel(OUTDIR / "Empirical_Stats_FeaturesOfInterest.xlsx", index=False)
stat_focus.to_csv(OUTDIR / "Empirical_Stats_FeaturesOfInterest.csv", index=False)


cat_features = [c for c in features_of_interest_cat if c in df.columns]
num_features = [c for c in features_of_interest_num if c in df.columns]
X_base = df[cat_features + num_features].copy()

if 'Order Date' in df.columns:
    cutoff = df['Order Date'].quantile(0.8)
    train_idx = df['Order Date'] <= cutoff
    test_idx  = df['Order Date'] >  cutoff
else:
    train_idx, test_idx = train_test_split(df.index, test_size=0.2, stratify=y, random_state=SEED)

X_train = X_base.loc[train_idx]
X_test  = X_base.loc[test_idx]
y_train = y.loc[train_idx]
y_test  = y.loc[test_idx]


major, minor = map(int, sklver.split('.')[:2])
if (major, minor) >= (1, 2):
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
else:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

pre = ColumnTransformer(
    transformers=[
        ('cat', ohe, cat_features),
        ('num', 'passthrough', num_features)
    ],
    remainder='drop'
)

xgb = XGBClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9,
    eval_metric='logloss', random_state=SEED
)

pipe = Pipeline([('pre', pre), ('clf', xgb)])
pipe.fit(X_train, y_train)


X_train_trans = pipe.named_steps['pre'].transform(X_train)

ohe_names = []
if len(cat_features) > 0:
    ohe_fitted = pipe.named_steps['pre'].named_transformers_['cat']
    ohe_names = list(ohe_fitted.get_feature_names_out(cat_features))
feature_names_final = ohe_names + num_features


explainer = shap.TreeExplainer(pipe.named_steps['clf'])
ns = min(5000, X_train_trans.shape[0])
shap_values = explainer.shap_values(X_train_trans[:ns])


mean_abs = np.abs(shap_values).mean(axis=0)
shap_imp_rows = []
if len(ohe_names) > 0:
    for base in cat_features:
        idxs = [i for i, fn in enumerate(ohe_names) if fn.startswith(f"{base}_")]
        if idxs:
            shap_imp_rows.append([base, float(mean_abs[idxs].sum())])
for j, fn in enumerate(feature_names_final):
    if fn in num_features:
        shap_imp_rows.append([fn, float(mean_abs[j])])

shap_imp_df = pd.DataFrame(shap_imp_rows, columns=['Feature', 'Mean|SHAP|']).sort_values('Mean|SHAP|', ascending=False)
shap_imp_df['SHAP_Rank'] = np.arange(1, len(shap_imp_df) + 1)
shap_imp_df.to_excel(OUTDIR / "SHAP_Importance_Aggregated.xlsx", index=False)
shap_imp_df.to_csv(OUTDIR / "SHAP_Importance_Aggregated.csv", index=False)t
shap_focus = shap_imp_df[shap_imp_df['Feature'].isin(features_of_interest)].copy()
shap_focus.to_excel(OUTDIR / "SHAP_Importance_FeaturesOfInterest.xlsx", index=False)
shap_focus.to_csv(OUTDIR / "SHAP_Importance_FeaturesOfInterest.csv", index=False)
# ---------------------------
merged_focus = (stat_focus[['Feature','Test','Statistic','EffectSize','EffectType','p_value','q_value']]
                .merge(shap_focus[['Feature','Mean|SHAP|','SHAP_Rank']], on='Feature', how='outer'))
merged_focus = merged_focus.sort_values(['q_value','SHAP_Rank'], na_position='last')
merged_focus.to_excel(OUTDIR / "Empirical_Eval_FeaturesOfInterest.xlsx", index=False)
merged_focus.to_csv(OUTDIR / "Empirical_Eval_FeaturesOfInterest.csv", index=False))
plt.figure(figsize=(7, 5))
topN = shap_imp_df.head(N_TOP_PLOT)['Feature'].tolist()
plt.barh(topN[::-1], shap_imp_df.set_index('Feature').loc[topN, 'Mean|SHAP|'][::-1].values)
plt.tight_layout()
plt.savefig(OUTDIR / f"SHAP_SummaryBar_Top{N_TOP_PLOT}.png", dpi=DPI)
plt.close()
shap.summary_plot(shap_values, X_train_trans[:ns], feature_names=feature_names_final, show=False, max_display=N_TOP_PLOT)
plt.tight_layout()
plt.savefig(OUTDIR / f"SHAP_Beeswarm_Top{N_TOP_PLOT}.png", dpi=DPI, bbox_inches='tight')
plt.close()
dep_outdir = OUTDIR / "dependence_plots"
dep_outdir.mkdir(exist_ok=True, parents=True)

dep_feats = ['TPT_per_Unit', 'LeadTime_Deviation', 'Weight_per_Unit', 'log_UnitQty',
             'carrier_origin_risk', 'route_cum_late_rate', 'route_bb_mean',
             'carrier_bb_mean', 'route_orders_last7d', 'route_roll10_Weight_q90',
             'congestion_trend', 'Weight_vsCarrierMean', 'seq_pos_norm']

for feat in dep_feats:
    if feat not in num_features:
        continue
    try:
        j = feature_names_final.index(feat)
        shap.dependence_plot(ind=j, shap_values=shap_values, features=X_train_trans[:ns],
                             feature_names=feature_names_final, show=False)
        plt.tight_layout()
        plt.savefig(dep_outdir / f"SHAP_Dependence_{feat}.png", dpi=DPI, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"[WARN] Could not create dependence plot for {feat}: {e}")

print(f"\nSaved outputs in: {OUTDIR.resolve()}")
print("Tables:\n - Empirical_Stats_Augmented.(xlsx/csv)"
      "\n - Empirical_Stats_FeaturesOfInterest.(xlsx/csv)"
      "\n - SHAP_Importance_Aggregated.(xlsx/csv)"
      "\n - SHAP_Importance_FeaturesOfInterest.(xlsx/csv)"
      "\n - Empirical_Eval_FeaturesOfInterest.(xlsx/csv)")
print(f"Figures:\n - SHAP_SummaryBar_Top{N_TOP_PLOT}.png"
      f"\n - SHAP_Beeswarm_Top{N_TOP_PLOT}.png"
      "\n - dependence_plots/*.png")


Augmented features detected: ['adaptive_sla', 'congestion_flag', 'disruption_flag', 'time_drift', 'OrderMonth', 'OrderWeek', 'OrderWeekday', 'is_weekend', 'Weight_per_Unit', 'TPT_per_Unit', 'LeadTime_Deviation', 'Weight_vsCarrierMean', 'Unit_vsCarrierMean', 'Carrier_Origin', 'Plant_Destination', 'log_Weight', 'log_UnitQty', 'congestion_trend', 'sla_risk_trend', 'disruption_trend', 'time_drift_abs', 'carrier_origin_risk', 'route_cum_late_rate', 'route_bb_mean', 'carrier_cum_late_rate', 'carrier_bb_mean', 'seq_pos_norm', 'interarrival_idx_gap', 'route_roll10_TPT_mean', 'route_roll10_TPT_std', 'route_roll10_TPT_q90', 'route_roll10_LeadTime_Deviation_mean', 'route_roll10_LeadTime_Deviation_std', 'route_roll10_LeadTime_Deviation_q90', 'route_roll10_Weight_mean', 'route_roll10_Weight_std', 'route_roll10_Weight_q90', 'route_orders_last7d', 'route_orders_last14d', 'route_orders_last30d']
Features of interest (present): ['Origin Port', 'Carrier', 'Plant Code', 'Destination Port', 'Plant_Destina

/var/folders/cm/x59s6wt92191zsp8lhjyn09r0000gn/T/ipykernel_61253/3808962942.py:243: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values, X_train_trans[:ns], feature_names=feature_names_final, show=False, max_display=N_TOP_PLOT)



Saved outputs in: /Users/dhadel/empirical_eval_outputs
Tables:
 - Empirical_Stats_Augmented.(xlsx/csv)
 - Empirical_Stats_FeaturesOfInterest.(xlsx/csv)
 - SHAP_Importance_Aggregated.(xlsx/csv)
 - SHAP_Importance_FeaturesOfInterest.(xlsx/csv)
 - Empirical_Eval_FeaturesOfInterest.(xlsx/csv)
Figures:
 - SHAP_SummaryBar_Top20.png
 - SHAP_Beeswarm_Top20.png
 - dependence_plots/*.png
